### Импортируем необходимые нам библиотеки

In [39]:
import numpy as np

# загрузим специальную библиотеку, позволяющую решать оптимизационные задачи линейного программирования
from scipy.optimize import linprog

Для целей поиска оптимального графика подачи тепла используем зависимость температуры внутри помещения от времени остывании и нагревании здания:

$$
\Large{t}_в = {t}_н + \frac{Q}{Vx} + \frac{({t^{'}}_в - {t}_н - \frac{Q}{Vx})}{e^{(\frac{z}{\gamma})}}
$$

- $\Large{t}_в$ - температура, которая установится через z часов после нарушения теплового режима;
- $\Large{t^{'}}_в$ - температура, которая была в помещении на момент нарушения теплового режима;
- $\Large{t}_н$ - температура наружного воздуха;
- Q - количество тепла, подаваемого в помещение, Дж/час;
- V - объем здания по наружному обмеру, $м^{3}$;
- x -  отопительная характеристика здания, $\Large\frac{Дж}{м^{3}ч^{o}C}$;
- $\Large\gamma$ - коэффициент аккумуляции, характеризующий аккумулирующую способность наружных ограждений, ч.

Задача состоит в том, чтобы, зная температурный профиль наружного воздуха на несколько периодов времени вперед, рассчитать график подачи тепла в эти периоды с минимумом суммы поданного тепла за эти периоды.

Математически это означает, что целевая функция оптимизационной задачи является следующей суммой:

$$
\Large{Q} = {Q}_0 + {Q}_1 + ..... + {Q}_n  --> min
$$
где:
- n - количество прогнозных периодов.

или в матричном виде:

$$
\Large с^{T}\hat{Q}  --> min
$$
где:
- $\Large c$ - единичный вектор размерности n;
- $\Large\hat{Q}$ - вектор значений подачи тепла в разные периоды времени размерности n.

Исходя из формулы, учитывающей инерцию здания в остывании и нагревании, к целевой функции необходимо добавить следующие ограничения на $\Large {Q}_i$:

$$
\Large t_{н i-1} + \frac{Q_{i-1}}{Vx} + \frac{(t_{i-1} - t_{н i-1} - \frac{Q_{i-1}}{Vx})}{e^{(\frac{z}{\gamma})}} = t_{i} >= 16
$$

или упростив:

$$
\Large t_{i-1}\alpha + t_{н i-1}(1-\alpha) + Q_{i-1}\beta >= 16
$$
$$
\Large t_{i-1}\alpha + Q_{i-1}\beta >= 16 - t_{н i-1}(1-\alpha)
$$

где:
- $\Large \alpha = \frac{1}{e^{(\frac{z}{\gamma})}}$
- $\Large \beta = \frac{1 - \alpha}{qV}$

В ограничениях на Qi-1 все еще присутствует значение температуры в предыдущий период ti-1.</br>
Однако это значение можно также выразить через значение ti-2 и Qi-2.</br>
Таким образом, можно рекурсивно дойти до известной нам температуры внутри помещения в нулевой период t0.</br>
После рекурсии до температуры t0 ограничения на Qi будут выглядеть следующим образом:

$$
\Large Q_{0}*\beta*\alpha^{i-1} + Q_{1}*\beta*\alpha^{i-2} + ... + Q_{i}*\beta*\alpha^{0} >= 16 - t_{0}*\alpha{i} - t_{н0}*(1-\alpha)*alpha^{i-1} - ... - t_{нi}*(1-\alpha)*alpha^{0}
$$

Или в матричном виде ограничения записываются как:

$$
\Large A*\hat{Q} >= \hat{b}
$$
где:
- $\Large\hat{Q}$ - вектор значений подачи тепла в прогнозные периоды времени;
- $\Large{A}$ - матрица ограничений вида $\begin{bmatrix} \beta & 0 & 0 & ... & 0 \\ \beta*\alpha & \beta & 0 & ... & 0 \\\ ... & .. & .. & ... & .. \\\\ \beta*\alpha^{n-1} & .. & .. & ... & \beta \end{bmatrix}$
- $\Large\hat{b}$ - вектор вида $\begin{bmatrix} t_{0}*\alpha + t_{н0}*(1-\alpha) \\ t_{0}*\alpha^{2} + t_{н0}*(1-\alpha)*\alpha + t_{н1}*(1-\alpha) \\\ ..................................... \\\\ t_{0}*\alpha^{n} + t_{н0}*(1-\alpha)*\alpha^{n-1} + ... t_{нn}*(1-\alpha)*alpha^{0} \end{bmatrix}$

Мы определили целевую функцию и ограничения на ее переменные в виде матрицы и вектора. Теперь мы можем решить задачу линейного программирования (оптимизация такого рода выбрана по причине линейности как целевой функции, так и ограничений)

### Зададим характеристики здания, начальную температуру внутри помещения и ограничение по ней

In [40]:
# характеристика здания
qV = 0.0739

# количество часов, необходимое зданию для остывания в e раз (2,71 раз)
gamma = 100

alpha = np.round(np.exp(-1/gamma),4)
betta = np.round((1 - alpha)/qV,4)

# начальная температура внутри помещения
t0 = 17.6
# нижняя граница внутренней температуры в здании
tmin = 16.5

In [41]:
alpha, betta

(0.99, 0.1353)

### Зададим прогноз температуры наружного воздуха:

In [42]:
outside_temperature = [-7,-1,0,-3, 0,-1,-8,-2,2,-3]
for i in range(10):
    print(f'tвнеш = {outside_temperature[i]} град.')

tвнеш = -7 град.
tвнеш = -1 град.
tвнеш = 0 град.
tвнеш = -3 град.
tвнеш = 0 град.
tвнеш = -1 град.
tвнеш = -8 град.
tвнеш = -2 град.
tвнеш = 2 град.
tвнеш = -3 град.


### Нахождение оптимального профиля подачи тепла

Для решения оптимизационной задачи, нам, используя характеристики здания, нач температуру и ограничение по температуре, необходимо найти вектор C, матрицу ограничений А и вектор ограничений b.

In [43]:
# вектор с - это просто единичный вектор
c = np.ones(10, dtype=int)
c

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [44]:
# создадим матрицу ограничеинй А
A =np.diag(np.full(10,-betta))

for i in range(1,10):
    A += np.diag(np.full(10-i,-betta*(alpha**i)), -i)

In [45]:
# матрица A получилась в необходимом нам виде
A

array([[-0.1353    ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.133947  , -0.1353    ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.13260753, -0.133947  , -0.1353    ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.13128145, -0.13260753, -0.133947  , -0.1353    ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.12996864, -0.13128145, -0.13260753, -0.133947  , -0.1353    ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.12866895, -0.12996864, -0.13128145, -0.13260753, -0.133947  ,
        -0.1353    ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.12738226, -0.12866895, -0.12996864, -0.13128145, -0.13260753,
        -0.133947  , -0.1353    ,  0.        

In [46]:
#создадим вектор ограничений b, сделаем его из двух частей, которые сложим вместе, так как это упростит создание циклов
b_1 = np.zeros(10)
b_1[0] = outside_temperature[0]*(1 - alpha)
for i in range(1,10):
    b_1[i] = b_1[i-1] + outside_temperature[i]*(1 - alpha)*alpha**i

In [47]:
#вторая часть вектора b
b_2 = np.zeros(10)
for i in range(10):
    b_2[i] = -tmin + t0*alpha**(i+1)

In [48]:
# складываем обе части вместе
b = np.round(b_1 + b_2, 2)
b

array([ 0.85,  0.67,  0.5 ,  0.3 ,  0.13, -0.05, -0.29, -0.47, -0.62,
       -0.8 ])

In [49]:
# проводим поиск оптимального решения
result = linprog(c, A_ub=A, b_ub=b)

In [50]:
Qsum = np.round(result.fun,2)
print(f'Итого подача тепла за весь прогнозный период: {Qsum} Гкал')

Итого подача тепла за весь прогнозный период: 6.02 Гкал


In [51]:
#посмотрим на вектор Q
Q = np.round(np.array(result.x),2)
Q

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.37, 1.78, 1.35, 1.14, 1.38])

In [52]:
# расчитаем при таком профиле подачи тепла, как ведет себя температура внутри помещения
internal_temp = []
internal_temp.append(t0)

for i in range(1,10):
    ti = internal_temp[i-1]*alpha + Q[i-1]*betta + outside_temperature[i-1]*(1-alpha)
    internal_temp.append(np.round(ti,1))

In [53]:
# также рассчитаем температуру теплоносителя в подающем трубопроводе после смешения
heat_temp = []

for i in range(10):
    ti = tmin + 72*(Q[i]/(qV*(tmin+40)))**0.8 + 15*(Q[i]/(qV*(tmin+40)))
    heat_temp.append(np.round(ti))

In [54]:
# выведем все результаты расчета
for i in range(10):
    print(f'Q{i} = {Q[i]} Гкал    tвнеш = {outside_temperature[i]}    tвнутр = {internal_temp[i]}    tпод = {heat_temp[i]}')

Q0 = 0.0 Гкал    tвнеш = -7    tвнутр = 17.6    tпод = 16.0
Q1 = 0.0 Гкал    tвнеш = -1    tвнутр = 17.4    tпод = 16.0
Q2 = 0.0 Гкал    tвнеш = 0    tвнутр = 17.2    tпод = 16.0
Q3 = 0.0 Гкал    tвнеш = -3    tвнутр = 17.0    tпод = 16.0
Q4 = 0.0 Гкал    tвнеш = 0    tвнутр = 16.8    tпод = 16.0
Q5 = 0.37 Гкал    tвнеш = -1    tвнутр = 16.6    tпод = 28.0
Q6 = 1.78 Гкал    tвнеш = -8    tвнутр = 16.5    tпод = 59.0
Q7 = 1.35 Гкал    tвнеш = -2    tвнутр = 16.5    tпод = 51.0
Q8 = 1.14 Гкал    tвнеш = 2    tвнутр = 16.5    tпод = 46.0
Q9 = 1.38 Гкал    tвнеш = -3    tвнутр = 16.5    tпод = 51.0
